In [36]:
import numpy as np
from scipy.stats import unitary_group
from numpy import linalg as LA
from scipy.linalg import block_diag
from scipy.linalg import logm, expm
import qiskit

In [37]:
#functions
def h(xx):
    x=np.real(xx)
    return -x*np.log2(x)-(1-x)*np.log2(1-x)
def H(v):
    summ=0
    d=len(v)
    for j in range(d):
        summ=summ+h(v[j])
    return summ
#following function rotates gamma0=diag(w) with U, extract blocks wrt Y=(2,2,1), and returns block spectrum v
def vvector(w,U):
    gamma0=np.diag(w)
    Udag=np.conjugate(U.T)
    gamma=np.matmul(U,np.matmul(gamma0,Udag))
    block1=gamma[0:2,0:2]
    block2=gamma[2:4,2:4]
    block1ev=LA.eigvals(block1)
    block2ev=LA.eigvals(block2)
    v=np.array([block1ev[0].real,block1ev[1].real,block2ev[0].real,block2ev[1].real,gamma[4][4].real])
    return(v)

#following function returns unitary epsilon close to identity
def unitaryclosetoid(epsilon):
    M = np.random.rand(d, d)-0.5 + 1j*np.random.rand(d, d)  # a (very basic) random complex matrix
    A = (M - np.conj(M.T)) #make it antihermitian. eigenvalues are like between -3i and 3i...
    deltaU=expm(epsilon*A) #unitary close to identity
    return(deltaU)

In [38]:
#some sorted w vectors for which a conjecture is there:

ws=[np.array([0.06107393, 0.09195662, 0.17570817, 0.31671073, 0.79044988]),
 np.array([0.17141052, 0.1879521 , 0.19878319, 0.25597978, 0.72284858]),
 np.array([0.11720914, 0.58447904, 0.62896595, 0.67608258, 0.83311845]),
 np.array([0.05643196, 0.07543735, 0.08430196, 0.18567166, 0.76125603]),
 np.array([0.48612207, 0.51768943, 0.59436303, 0.61219746, 0.96326387]),
 np.array([0.01507189, 0.06700813, 0.09221116, 0.35646417, 0.67562343]),
 np.array([0.05992716, 0.11306935, 0.13708026, 0.27367042, 0.86992894]),
 np.array([0.03294419, 0.55511419, 0.82555108, 0.88060291, 0.99802625]),
 np.array([0.10149043, 0.40318086, 0.69534272, 0.69545063, 0.77544863]),
 np.array([0.10388813, 0.4801112 , 0.62125352, 0.6466061 , 0.85106515]),
 np.array([0.12499359, 0.394975  , 0.47638738, 0.52556822, 0.65358446]),
 np.array([0.07702118, 0.11036463, 0.20356913, 0.30515876, 0.67132918]),
 np.array([0.17428388, 0.37172466, 0.382868  , 0.38380627, 0.86961459])]
 

#choose w vector
#w=np.array([0.7,0.02,0.46,0.46,0.72])
w=np.sort(ws[0])
t=sum(w)
d=len(w)
haveconjecture=True
if t/5<(w[1]+w[2])/2:
    vconj=np.array([(w[1]+w[2])/2,(w[1]+w[2])/2,(w[0]+w[3]+w[4])/3,(w[0]+w[3]+w[4])/3,(w[0]+w[3]+w[4])/3,])
    Iconj=H(vconj)
    print("1st regime")
elif t/5>(w[2]+w[3])/2:
    vconj=np.array([(w[2]+w[3])/2,(w[2]+w[3])/2,(w[0]+w[1]+w[4])/3,(w[0]+w[1]+w[4])/3,(w[0]+w[1]+w[4])/3,])
    Iconj=H(vconj)
    print("2nd regime")
else:
    haveconjecture=False
    print("unknown regime")

2nd regime


In [39]:
#number of global unitaries
m=500000
#number of best global unitaries to keep on the 'podium'
mpodium=30
#number of close-to-identity unitaries
mm=100000
#decide how close to identity
epsilon=0.001 #try carefully!
#later inspection
mmm=100000

#allocate memory for some relevant variables
#gamma0=np.diag(w)
vbest=np.array([w[j] for j in range(d)])
v=np.zeros(d)
Ibest=0.
I=0.
Ubest=np.identity(d,dtype=complex)
U=np.identity(d,dtype=complex)
podium=[[0.,np.identity(d,dtype=complex)] for i in range(mpodium)] #will be filled with couples (I value, U)

#global optimization
for k in range(m):
    U=unitary_group.rvs(d)
    v=vvector(w,U)
    I=H(v)
    if I>podium[0][0]:
        podium[0][0]=np.copy(I)
        podium[0][1]=np.copy(U)
        #order podium; best (I,U) is given by podium[-1]
        podium = sorted(podium, key=lambda x: x[0])
                               
Ibestbeforelocal=podium[-1][0]

#local optimization for each unitary on the podium
deltaU=np.identity(d,dtype=complex)

#!!! watch out! some a-posteriori check of UNITARITY OF Ubest should be done!!
#Bc we are multiplying more and more unitaries... (how many?)
#the errors add up, as opposed to what happens in the global maximization

for k in range(mm):
    deltaU=unitaryclosetoid(epsilon)
    for l in range(mpodium): #treat  podium couples  (I,U) independently
        U = np.matmul(podium[l][1],deltaU) #unitary close to podium[l][1]
        v=vvector(w,U)
        I=H(v)
        if I>podium[l][0]:
            podium[l][0]=np.copy(I)
            podium[l][1]=np.copy(U)
            
#reorder podium            
podium = sorted(podium, key=lambda x: x[0])
# best among podium
Ibest=np.copy(podium[-1][0])
Ubest=np.copy(podium[-1][1])

In [40]:
print("estimated Imax value after global minimization: ",Ibestbeforelocal)
print("estimated Imax value after local minimization: ",Ibest)
print("upper bound: ", d*h(t/d))

estimated Imax value after global minimization:  4.295699875739064
estimated Imax value after local minimization:  4.2963585681080465
upper bound:  4.325251880106437


In [41]:
#unitarity check... ok ok it's fine at this stage
Ubestdag=np.conjugate(Ubest.T)
np.matmul(Ubest,Ubestdag).round(14)

array([[ 1.+0.j,  0.+0.j, -0.+0.j,  0.-0.j,  0.+0.j],
       [ 0.-0.j,  1.+0.j, -0.+0.j,  0.+0.j,  0.+0.j],
       [-0.-0.j, -0.-0.j,  1.+0.j, -0.-0.j,  0.-0.j],
       [ 0.+0.j,  0.-0.j, -0.+0.j,  1.+0.j,  0.-0.j],
       [ 0.-0.j,  0.-0.j,  0.+0.j,  0.+0.j,  1.-0.j]])

In [42]:
count=0
for j in range(mpodium):
    if Ibest-podium[j][0]<0.0001:
        count=count+1
print("Out of ", mpodium, " couples (I,U) on the podium, the first ",count, " couples are close to the winner couple")

Out of  30  couples (I,U) on the podium, the first  1  couples are close to the winner couple


In [43]:
#assuming the winner Ubest is actually good, one can push the local maximization even further to get more digits
#unitarity check in the end could be more important than before as you multiply more and more unitaries

#mmm=10000
epsilon=0.00001

for k in range(mmm):
    deltaU=unitaryclosetoid(epsilon)
    U = np.matmul(Ubest,deltaU) #unitary close to Ubest
    v=vvector(w,U)
    I=H(v)
    if I>Ibest:
        Ibest=np.copy(I)
        Ubest=np.copy(U)

In [44]:
print("estimated Imax value after further local minimization: ",Ibest)
print("upper bound: ", d*h(t/d))

estimated Imax value after further local minimization:  4.296358740935817
upper bound:  4.325251880106437


In [45]:
#unitarity check...also fine
Ubestdag=np.conjugate(Ubest.T)
np.matmul(Ubest,Ubestdag).round(14)

array([[ 1.+0.j,  0.+0.j, -0.+0.j,  0.-0.j,  0.+0.j],
       [ 0.-0.j,  1.+0.j, -0.+0.j,  0.+0.j,  0.+0.j],
       [-0.-0.j, -0.-0.j,  1.+0.j, -0.-0.j,  0.-0.j],
       [ 0.+0.j,  0.-0.j, -0.+0.j,  1.+0.j,  0.-0.j],
       [-0.-0.j,  0.-0.j,  0.+0.j,  0.+0.j,  1.-0.j]])

In [46]:
#check conjecture, if any
if haveconjecture==True:
    print("estimated Imax value after further local minimization: ",Ibest)
    print("conjectured value: ",Iconj)
    diff=Iconj-Ibest
    if diff<0:
        print("Conjecture is wrong!")
    else:
        print("Conjecture could be true: difference is ", diff)
vbest=np.sort(vvector(w,Ubest))
vconj=np.sort(vconj)
print("\n vbest ",vbest.round(6))
print("vconj ",vconj.round(6))

estimated Imax value after further local minimization:  4.296358740935817
conjectured value:  4.305307034935393
Conjecture could be true: difference is  0.008948293999575974

 vbest  [0.2237   0.256072 0.313807 0.31803  0.324291]
vconj  [0.246209 0.246209 0.314493 0.314493 0.314493]


In [35]:
##########################################################

In [13]:
gamma=np.matmul(np.matmul(Ubest,np.diag(w)),Ubestdag)
block1=gamma[0:2,0:2]
block2=gamma[2:4,2:4]
block1ev=LA.eigvals(block1)
block2ev=LA.eigvals(block2)
v=np.array([block1ev[0].real,block1ev[1].real,block2ev[0].real,block2ev[1].real,gamma[4][4].real])
t=sum(w)
H(v)

4.292993663264236